In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = np.loadtxt('c:/users/student/desktop/stock_daily.csv',delimiter=',',dtype=np.float32)

In [3]:
def MinMaxScaler(data):
    bunja = data - np.min(data,0) #axis=0 이 행임
    bunmo = np.max(data,0) - np.min(data, 0)
    return bunja/bunmo+1e-10

In [4]:
import random

In [5]:
# data = MinMaxScaler(data)

In [6]:
# x_data = data
# y_data = data[:,[-1]]

In [11]:
# idx = random.sample(range(len(data)), int(len(data)*0.8))
# train_x = x_data[idx,:]
# train_y = y_data[idx,:]
# test_x = np.where(x_data)
# test_y = y_data[-y_data[idx],:]

SyntaxError: invalid syntax (<ipython-input-11-5775fd0f8747>, line 4)

In [35]:
# train_x = MinMaxScaler(train_x)
# train_x = train_x[::-1] #열과 로우 거꾸로 뒤집기!

In [39]:
train_x.shape

(585, 4)

In [37]:
seq_length = 7
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01


xy = np.loadtxt('stock_daily.csv', delimiter=',')
xy = xy[::-1] # 순서를 뒤집고
xy = MinMaxScalar(xy) # 스케일을 맞추고
x = xy
y = xy[:,[-1]]
dataX = []
dataY = []
for i in range(0, len(y)-seq_length):
    _x = x[i:i+seq_length]
    _y = y[i+seq_length]
    dataX.append(_x)
    dataY.append(_y)

train_size = int(len(dataY)*0.7)
test_size = len(dataY)- train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

array([[0.09019206, 0.11594636, 0.10839645, 0.14979833],
       [0.08791915, 0.09516431, 0.09906515, 0.09108184],
       [0.6537744 , 0.6561279 , 0.6741668 , 0.10814735],
       ...,
       [0.40724063, 0.40613645, 0.40605274, 0.15959488],
       [0.7091425 , 0.71341586, 0.72716165, 0.10077978],
       [0.839549  , 0.82897455, 0.82998216, 0.15705834]], dtype=float32)

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, shape=[None,1])
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, 
                                   activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell,X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim, 
                                          activation_fn=None)
loss = tf.reduce_mean(tf.reduce_sum(tf.square(Y_pred-Y)))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
    l,_ = sess.run([loss, train], feed_dict={X:dataX, Y:dataY})
    if step % 100 == 0:
        print ("step:",step,"loss:",l)


In [12]:
#rmse

target = tf.placeholder(tf.float32,[None,1])
prediction = tf.placeholder(tf.float32,[None,1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(target-prediction)))

In [ ]:
test_prediction = sess.run(Y_pred, feed_dict=[X:testX])
print("rmse :", sess.run(rmse, feed_dict=[target:testY, prediction:test_prediction]))

In [ ]:
plt.plot(testY)
plt.plot(test_prediction)